# Instruction Finetuning

In this script, we investigate the usage of finetuning on the *UNSW-NB15* dataset using various preprocessing techniques.

# Key-Value pairs Text Encoding

In [1]:
from datasets import load_dataset, Dataset
Dataset.cleanup_cache_files
from dotenv import load_dotenv
from os import getenv
import os

load_dotenv()
HUGGING_FACE_READ_TOKEN = getenv("HUGGING_FACE_READ_TOKEN")

dataset = load_dataset("Jetlime/NF-UNSW-NB15-v2", streaming=False, split="train")

/home/paul/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using the latest cached version of the dataset since Jetlime/NF-UNSW-NB15-v2 couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /home/paul/.cache/huggingface/datasets/Jetlime___nf-unsw-nb15-v2/default/0.0.0/e787691e196b078564cfc32297f511298a45a15f (last modified on Wed May 22 12:23:27 2024).
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
import gc

import torch
import mlflow
from datasets import load_dataset
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)
from trl import KTOConfig, KTOTrainer, setup_chat_format

mlflow.set_tracking_uri(uri="http://127.0.0.1:8080")
mlflow.set_experiment(experiment_name="Testing finetuning")

<Experiment: artifact_location='mlflow-artifacts:/967304590420382862', creation_time=1715755222120, experiment_id='967304590420382862', last_update_time=1715755222120, lifecycle_stage='active', name='Testing finetuning', tags={}>

In [3]:
if torch.cuda.get_device_capability()[0] >= 8:
    attn_implementation = "flash_attention_2"
    torch_dtype = torch.bfloat16
else:
    attn_implementation = "eager"
    torch_dtype = torch.float16

In [4]:
# Model
base_model = "meta-llama/Meta-Llama-3-8B-Instruct"
new_model = "OrpoLlama-3-8B"

# QLoRA config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch_dtype,
    bnb_4bit_use_double_quant=True,
)

# LoRA config
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['up_proj', 'down_proj', 'gate_proj', 'k_proj', 'q_proj', 'v_proj', 'o_proj']
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, token=HUGGING_FACE_READ_TOKEN)

# Load model
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    attn_implementation=attn_implementation,
    token=HUGGING_FACE_READ_TOKEN
)
model, tokenizer = setup_chat_format(model, tokenizer)
model = prepare_model_for_kbit_training(model)

/home/paul/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
`low_cpu_mem_usage` was None, now set to True since model is quantized.
Loading checkpoint shards: 100%|██████████| 4/4 [00:04<00:00,  1.17s/it]


In [5]:
# Use only a small subset of the training set for a first finetuning trial
dataset = dataset.train_test_split(test_size=0.95, seed=123, stratify_by_column="Attack")
dataset_finetuning = dataset["train"]
dataset_finetuning

Dataset({
    features: ['input', 'output', 'Attack'],
    num_rows: 113538
})

In [6]:
import random

def format_chat_template(row):
    row['prompt'] = row["input"]
    if random.randrange(0,1):
        row["label"] = False
        if row["output"] == 1:
            row["completion"] = '0'
        else:
            row["completion"] = '1'
    else:
        row["label"] = True
        row["completion"] = str(row["output"])
    return row

dataset_finetuning = dataset_finetuning.map(
    format_chat_template, num_proc=os.cpu_count()
)
dataset_finetuning

Map (num_proc=16): 100%|██████████| 113538/113538 [00:01<00:00, 100567.04 examples/s]


Dataset({
    features: ['input', 'output', 'Attack', 'prompt', 'label', 'completion'],
    num_rows: 113538
})

In [7]:
training_args = KTOConfig(
    beta=0.1,
    desirable_weight=1.0,
    undesirable_weight=1.0,
    output_dir="./results-KTO/"
)

kto_trainer = KTOTrainer(
    model,
    args=training_args,
    train_dataset=dataset_finetuning,
    tokenizer=tokenizer,
)

kto_trainer.train()
kto_trainer.save_model(new_model)

/home/paul/.local/lib/python3.10/site-packages/trl/trainer/kto_trainer.py:417: UserWarning: When using DPODataCollatorWithPadding, you should set `max_length` in the KTOTrainer's init it will be set to `512` by default, but you should do it yourself in the future.
  warnings.warn(
/home/paul/.local/lib/python3.10/site-packages/trl/trainer/kto_trainer.py:427: UserWarning: When using DPODataCollatorWithPadding, you should set `max_prompt_length` in the KTOTrainer's init it will be set to `128` by default, but you should do it yourself in the future.
  warnings.warn(
/home/paul/.local/lib/python3.10/site-packages/trl/trainer/kto_trainer.py:457: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your KTOConfig we have set it for you, but you should do it yourself in the future.
  warnings.warn(
Filtering undesirable examples: 100%|██████████| 113538/113538 [01:36<00:00, 1170.86 examples/s]


ZeroDivisionError: float division by zero

: 

In [ ]:
# Flush memory
del trainer, model
gc.collect()
torch.cuda.empty_cache()

# Reload tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(base_model)
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="auto",
)
model, tokenizer = setup_chat_format(model, tokenizer)

# Merge adapter with base model
model = PeftModel.from_pretrained(model, new_model)
model = model.merge_and_unload()